In [3]:
!pip -q install xgboost==2.0.3 flask==3.0.3 flask-ngrok==0.0.25 joblib==1.4.2
!pip install -U scikit-learn xgboost flask flask-ngrok joblib nltk




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully

In [5]:
!pip install -U xgboost==2.1.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 3.1.1
    Uninstalling xgboost-3.1.1:
      Successfully uninstalled xgboost-3.1.1


In [8]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# GPU config for XGBoost ≥ 2.0
device_type = 'cuda'   # use GPU
tree_method = 'hist'   # efficient histogram algorithm

**Import + define pipeline**

In [9]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews
import pandas as pd

docs = []
labels = []
for cat in movie_reviews.categories():  # 'pos' | 'neg'
    for fid in movie_reviews.fileids(cat):
        docs.append(movie_reviews.raw(fid))
        labels.append(1 if cat == 'pos' else 0)

df = pd.DataFrame({'review': docs, 'sentiment': labels})
df.sample(5, random_state=42)
print(df['sentiment'].value_counts())


sentiment
0    1000
1    1000
Name: count, dtype: int64


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


**Train/ Test Split**

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['sentiment'],
    test_size=0.2, random_state=42, stratify=df['sentiment']
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


Train size: 1600
Test size: 400


**Using xgboost ( XGBClassifier)**

In [14]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# GPU configuration for XGBoost ≥ 2.0
device_type = 'cuda'   # Use GPU
tree_method = 'hist'   # Efficient histogram algorithm

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=30000,
        ngram_range=(1, 2),
        stop_words='english',
        lowercase=True,
        strip_accents='unicode'      # ✅ Text cleaning
    )),
    ('clf', XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        device=device_type,
        tree_method=tree_method,
        random_state=42
    ))
])


**Applying Grid Search(defined parameters)**

In [15]:
param_grid = {
    'tfidf__max_features': [15000, 30000],
    'clf__n_estimators': [200, 400, 600],
    'clf__max_depth': [4, 6, 8],
    'clf__learning_rate': [0.05, 0.1],
    'clf__subsample': [0.8, 1.0],
    'clf__colsample_bytree': [0.8, 1.0],
    'clf__scale_pos_weight': [1]   # adjust later if class imbalance occurs
}


In [ ]:
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='f1',
    n_jobs=-1,      # use all cores
    cv=3,           # 3-fold cross-validation
    verbose=1
)

grid.fit(X_train, y_train)

print("✅ Best F1 (CV):", grid.best_score_)
print("✅ Best Params:", grid.best_params_)


Fitting 3 folds for each of 144 candidates, totalling 432 fits


**Conclusion Matrix**

In [ ]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print("✅ Test Accuracy:", accuracy_score(y_test, y_pred))
print("✅ Test F1:", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=3))


In [12]:
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))


Test Accuracy: 0.53
Test F1: 0.15315315315315314
              precision    recall  f1-score   support

           0      0.516     0.975     0.675       200
           1      0.773     0.085     0.153       200

    accuracy                          0.530       400
   macro avg      0.644     0.530     0.414       400
weighted avg      0.644     0.530     0.414       400



/usr/local/lib/python3.12/dist-packages/xgboost/core.py:158: UserWarning: [18:35:42] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


***Save for flask web app(model saving)***

In [13]:
import joblib, os

os.makedirs("artifacts", exist_ok=True)
joblib.dump(best_model, "artifacts/sentiment_xgb_pipeline.joblib")
print("Model saved at artifacts/sentiment_xgb_pipeline.joblib")


Model saved at artifacts/sentiment_xgb_pipeline.joblib
